# SV-SCN Training on Google Colab

This notebook trains the Single-View Shape Completion Network on Google Colab's free GPU.

**Steps:**
1. Setup environment & install dependencies
2. Clone/upload project code
3. Generate placeholder training data
4. Train the model
5. Download trained checkpoint

**Runtime:** ~2-3 hours for full training (150 epochs, 500 samples)

## Step 1: Check GPU & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

## Step 2: Clone Project (or Upload Files)

In [ ]:
# Option A: Clone from GitHub (replace with your repo URL)
# !git clone https://github.com/yourusername/frozo-3d-model.git
# %cd frozo-3d-model

# Option B: Upload project folder manually
# Use Colab's file browser to upload the entire frozo-3d-model folder

# For now, let's create the project structure
import os
os.makedirs('frozo-3d-model', exist_ok=True)
%cd frozo-3d-model

# You'll need to upload these folders:
# - svscn/
# - scripts/
# - requirements.txt

print("\n==> Upload your project files using the file browser on the left")
print("    Or clone from GitHub if you have it in a repo")

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install open3d>=0.17.0
!pip install trimesh>=4.0.0
!pip install scipy>=1.10.0
!pip install objaverse>=0.1.7
!pip install tensorboard>=2.14.0
!pip install pytest>=7.4.0

print("\n✅ All dependencies installed!")

## Step 4: Generate Placeholder Data

This creates synthetic furniture meshes for training/testing.

In [ ]:
# Generate placeholder ShapeNet-like data
!python -m svscn.data.shapenet \
    --placeholder \
    --output_dir data/shapenet \
    --samples_per_category 167

print("\n✅ Generated 500 placeholder furniture meshes")

In [ ]:
# Preprocess: Mesh → Point Cloud
!python -m svscn.data.preprocess \
    --input_dir data/shapenet \
    --output_dir data/processed \
    --num_points 8192

print("\n✅ Converted meshes to point clouds")

In [ ]:
# Generate training pairs (partial + full)
!python -m svscn.data.augment \
    --input_dir data/processed \
    --output_dir data/training \
    --views 3

print("\n✅ Generated training pairs (partial + complete point clouds)")

## Step 5: Quick Test (Optional)

Test the model architecture before training.

In [ ]:
# Quick model test
import torch
from svscn.models import SVSCN

model = SVSCN(num_classes=3)
print(f"Model created successfully")

# Test forward pass
batch_size = 4
partial = torch.randn(batch_size, 2048, 3)
class_ids = torch.randint(0, 3, (batch_size,))

if torch.cuda.is_available():
    model = model.cuda()
    partial = partial.cuda()
    class_ids = class_ids.cuda()

completed = model(partial, class_ids)

print(f"\nInput shape: {partial.shape}")
print(f"Output shape: {completed.shape}")
print(f"✅ Model test passed!")

## Step 6: Train Model

### Quick Training (10 epochs, ~5-10 min)
For testing the pipeline:

In [ ]:
# Quick training for testing
!python scripts/train.py \
    --data_dir data/training \
    --epochs 10 \
    --batch_size 32 \
    --checkpoint_dir checkpoints \
    --log_dir logs \
    --device cuda

### Full Training (150 epochs, ~2-3 hours)
For production model:

In [ ]:
# Full training (uncomment to run)
# WARNING: This will take 2-3 hours

!python scripts/train.py \
    --data_dir data/training \
    --epochs 150 \
    --batch_size 32 \
    --checkpoint_dir checkpoints \
    --log_dir logs \
    --device cuda

## Step 7: Monitor Training (TensorBoard)

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

## Step 8: Test Inference

In [ ]:
# Create a test partial cloud
import numpy as np

# Generate random partial cloud
partial = np.random.randn(2048, 3).astype(np.float32)
partial = (partial - partial.mean(axis=0)) / partial.std()

np.save('test_partial.npy', partial)
print(f"Created test partial cloud: {partial.shape}")

In [ ]:
# Run inference
!python scripts/infer.py \
    --checkpoint checkpoints/best.pt \
    --input test_partial.npy \
    --output completed.npy \
    --class_id 0 \
    --device cuda

print("\n✅ Inference complete!")

## Step 9: Visualize Results (Optional)

In [ ]:
# Visualize using matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

partial = np.load('test_partial.npy')
completed = np.load('completed.npy')

fig = plt.figure(figsize=(15, 5))

# Partial cloud
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1)
ax1.set_title('Input (Partial)')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

# Completed cloud
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1)
ax2.set_title('Output (Completed)')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')

# Both overlaid
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(partial[:, 0], partial[:, 1], partial[:, 2], c='blue', s=1, alpha=0.5, label='Partial')
ax3.scatter(completed[:, 0], completed[:, 1], completed[:, 2], c='green', s=1, alpha=0.3, label='Completed')
ax3.set_title('Overlay')
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_zlabel('Z')
ax3.legend()

plt.tight_layout()
plt.show()

print(f"Partial points: {len(partial)}")
print(f"Completed points: {len(completed)}")

## Step 10: Download Trained Model

In [ ]:
# Download the best checkpoint to your local machine
from google.colab import files

# Download checkpoint
files.download('checkpoints/best.pt')

print("\n✅ Model downloaded! You can now use it locally for inference.")

## Step 11: Export as Mesh (Optional)

In [ ]:
# Export as GLB for AR
!python scripts/infer.py \
    --checkpoint checkpoints/best.pt \
    --input test_partial.npy \
    --output completed.glb \
    --export_mesh \
    --class_id 0 \
    --device cuda

# Download GLB file
files.download('completed.glb')

print("\n✅ GLB file downloaded! You can view it in 3D viewers or AR apps.")

## Summary

You've successfully:
1. ✅ Set up the environment on Colab GPU
2. ✅ Generated placeholder training data (500 samples)
3. ✅ Trained the SV-SCN model
4. ✅ Tested inference
5. ✅ Downloaded the trained checkpoint

### Next Steps:
- Use the downloaded `best.pt` checkpoint locally for inference
- For production: Train on real ShapeNet/Objaverse data (5k-10k samples)
- Export models as GLB for web/mobile AR applications

### Local Inference (after downloading checkpoint):
```bash
python scripts/infer.py \
    --checkpoint best.pt \
    --input your_partial.npy \
    --output result.glb \
    --export_mesh \
    --device cpu
```